In [1]:
#Set environment options
import pandas as pd
# settings to display all columns
pd.set_option("display.max_columns", None)

In [2]:
#function to combine matchup data
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        #pass
        return None
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result




In [5]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
import time

#get nba team data including id's
nba_teams = pd.DataFrame(teams.get_teams())
#Create list of NBA team id's
team_ids = nba_teams['id'].tolist()

#season data should be pulled for 
s = '2022-23'


#function to call api and get all NBA games for a given season
def get_all_games(team__ids,season):
    team_season = []
    for i in team_ids:
        games = leaguegamefinder.LeagueGameFinder(team_id_nullable = i,season_nullable = season,season_type_nullable ='Regular Season').get_data_frames()[0]
        team_season.append(games)
        nba_season = pd.concat(team_season)
        time.sleep(.4)
    return(nba_season)

nba_season = get_all_games(team_ids,s)
nba_season = nba_season.sort_values(by=['GAME_DATE','GAME_ID'])
nba_season.head()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
home_team = nba_season[nba_season.MATCHUP.str.contains(' vs. ')]
away_team = nba_season[nba_season.MATCHUP.str.contains(' @ ')]

home_team_1 = home_team[['SEASON_ID','GAME_ID','GAME_DATE','TEAM_ID','TEAM_ABBREVIATION','MATCHUP','WL']]
home_team_1= pd.get_dummies(home_team_1, columns=['TEAM_ABBREVIATION','WL'], prefix=['HOME','HOME'])
home_team_1 = home_team_1.drop(['HOME_L'],axis=1)
home_team_1 = home_team_1.rename(columns={'TEAM_ID':'HOME_TEAM_ID','HOME_W':'HomeWIN = 1'})


away_team_1 = away_team[['GAME_ID','TEAM_ID','TEAM_ABBREVIATION']]
away_team_1 =  pd.get_dummies(away_team_1, columns=['TEAM_ABBREVIATION'], prefix=["AWAY"])

In [36]:
#Get home team game stats
from nba_api.stats.endpoints import teamdashboardbyopponent

call = teamdashboardbyopponent.TeamDashboardByOpponent(team_id=1610612757,date_to_nullable='2022-12-12')
overall_team_dashboard = call.get_data_frames()[3]
#conference_team_dashboard = call.get_data_frames()[1]
#division_team_dashboard = call.get_data_frames()[2]
#opponent_team_dashboard = call.get_data_frames()[3]

overall_team_dashboard.head()
#conference_team_dashboard.head()
#division_team_dashboard.head()
#opponent_team_dashboard.head()

,GROUP_SET,GROUP_VALUE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,vs. Opponent,Brooklyn Nets,2,0,2,0.0,96.0,71,172,0.413,25,69,0.362,37,49,0.755,19,62,81,49,23.0,13,7,7,38,48,204,-16.0,2,14,19,14,2,7,4,18,5,4,13,7,4,15,6,5,6,4,13,4,6,14,15,2,7,15
1,vs. Opponent,Charlotte Hornets,1,1,0,1.0,48.0,40,75,0.533,11,34,0.324,14,25,0.560,8,37,45,28,18.0,5,4,3,19,22,105,10.0,8,3,1,1,9,14,19,1,16,11,17,17,14,19,19,8,11,10,11,16,10,2,7,12,18,6
2,vs. Opponent,Cleveland Cavaliers,1,0,1,0.0,48.0,35,78,0.449,13,31,0.419,13,13,1.000,11,21,32,28,17.0,9,4,4,24,14,96,-18.0,8,14,9,14,9,19,18,14,11,16,6,18,19,1,12,19,19,10,9,9,10,3,11,19,19,18
3,vs. Opponent,Dallas Mavericks,1,0,1,0.0,48.0,36,80,0.450,13,31,0.419,27,35,0.771,12,32,44,20,12.0,6,2,2,21,27,112,-5.0,8,14,9,14,9,17,16,13,11,16,6,9,9,11,10,15,14,17,4,13,17,1,9,9,13,11
4,vs. Opponent,Denver Nuggets,2,1,1,0.5,96.0,92,174,0.529,32,75,0.427,39,47,0.830,22,61,83,53,23.0,13,9,6,35,44,255,24.0,2,3,9,10,2,2,2,4,2,2,3,4,7,3,3,6,5,2,13,4,4,11,14,3,3,2


In [42]:
'''function to get team stats per game and vs_Opponent stats per game for each home team'''
from nba_api.stats.endpoints import teamdashboardbyopponent
#create a tuple list of game dates and home team id's to pass to the api call
gameDate_homeTeam = list(zip(home_team_1.GAME_DATE, home_team_1.HOME_TEAM_ID))

def team_stats_pergame(gameDate_Team):
    #create an empty dataframe to hold the results
    teamOverall_stats = pd.DataFrame()
    #create an empty dataframe to hold vs_Opponent results
    teamOpponent_stats = pd.DataFrame()
    #create an empty dataframe to hold the concatination of the two data frames
    #team_stats = pd.DataFrame()
    

    #for row in gameDate_homeTeam:
    for tuple_item in gameDate_Team:
        #assign the game date and team id to variables to pass to the api call as parameters called dt and team_id
        dt = tuple_item[0]  
        team_id = tuple_item[1]
        #call the api and pass the parameters
        call = teamdashboardbyopponent.TeamDashboardByOpponent(team_id= team_id,date_to_nullable= dt)
        #get the data frame from the api call
        overall_data = call.get_data_frames()[0]
        #get the vs_Opponent data frame from the api call
        opponent_data = call.get_data_frames()[3]
        #concat the overall dataframe into the teamOverall_stats dataframe
        teamOverall_stats = pd.concat([teamOverall_stats,overall_data])
        #concat the vs_Opponent dataframe into the teamOpponent_stats dataframe
        teamOpponent_stats = pd.concat([teamOpponent_stats,opponent_data])
        
        #sleep for 1 second to avoid api call limit
        time.sleep(.4)
    return(teamOverall_stats,teamOpponent_stats)

#call the function and pass the gameDate_homeTeam list
team_Stats,Opp_stats = team_stats_pergame(test_10)

In [58]:
#add _team to the column names of the team_Stats dataframe
team_Stats.columns = [str(col) + '_team' for col in team_Stats.columns]
#add _Opp to the column names of the Opp_stats dataframe
Opp_stats.columns = [str(col) + '_Opp' for col in Opp_stats.columns]
#reset the index of the team_Stats dataframe
team_Stats = team_Stats.reset_index(drop=True)
#reset the index of the Opp_stats dataframe
Opp_stats = Opp_stats.reset_index(drop=True)
#reset the index of the homeTeam1_10 dataframe
homeTeam1_10 = homeTeam1_10.reset_index(drop=True)
#get the column names of all three dataframes and combine them into a list
team_Stats_cols = team_Stats.columns.tolist()
Opp_stats_cols = Opp_stats.columns.tolist()
homeTeam1_10_cols = homeTeam1_10.columns.tolist()
#combine the three lists into one list
cols = homeTeam1_10_cols + team_Stats_cols + Opp_stats_cols
#concat the team_Stats,Opp_stats and homeTeam1_10 dataframes on column axis to create the final dataframe
homeTeam_full = pd.concat([homeTeam1_10,team_Stats,Opp_stats],axis=1,ignore_index=True)
#assign the column names to the final dataframe
homeTeam_full.columns = cols

In [45]:
#get fist 10 rows of home_team_1
homeTeam1_10 = home_team_1.head(10)

In [53]:
print(team_Stats.columns)
print(Opp_stats.columns)
print(homeTeam1_10.columns)


Index(['GROUP_SET_team', 'GROUP_VALUE_team', 'GP_team', 'W_team', 'L_team',
       'W_PCT_team', 'MIN_team', 'FGM_team', 'FGA_team', 'FG_PCT_team',
       'FG3M_team', 'FG3A_team', 'FG3_PCT_team', 'FTM_team', 'FTA_team',
       'FT_PCT_team', 'OREB_team', 'DREB_team', 'REB_team', 'AST_team',
       'TOV_team', 'STL_team', 'BLK_team', 'BLKA_team', 'PF_team', 'PFD_team',
       'PTS_team', 'PLUS_MINUS_team', 'GP_RANK_team', 'W_RANK_team',
       'L_RANK_team', 'W_PCT_RANK_team', 'MIN_RANK_team', 'FGM_RANK_team',
       'FGA_RANK_team', 'FG_PCT_RANK_team', 'FG3M_RANK_team', 'FG3A_RANK_team',
       'FG3_PCT_RANK_team', 'FTM_RANK_team', 'FTA_RANK_team',
       'FT_PCT_RANK_team', 'OREB_RANK_team', 'DREB_RANK_team', 'REB_RANK_team',
       'AST_RANK_team', 'TOV_RANK_team', 'STL_RANK_team', 'BLK_RANK_team',
       'BLKA_RANK_team', 'PF_RANK_team', 'PFD_RANK_team', 'PTS_RANK_team',
       'PLUS_MINUS_RANK_team'],
      dtype='object')
Index(['GROUP_SET_Opp', 'GROUP_VALUE_Opp', 'GP_Opp', 'W_Op

In [16]:
gd,ht = gameDate_homeTeam[1]
print(ht)  
print(gd)

1610612744
2022-10-18
